# 1) Mount drive, unzip data, clone repo, install packages

## 1.1) Mount Drive and define paths
Run provided colab code to mount Google Drive. Then define dataset paths relative to mount point.

In [ ]:
!rm -rf '/content/sample_data'
!rm -rf '/content/*.jpg'
!rm -rf '/content/*.png'
!rm -rf '/content/*.json'

In [ ]:
# noinspection PyUnresolvedReferences,PyPackageRequirements
from google.colab import drive

mount_root_abs = '/content/drive'
drive.mount(mount_root_abs)
drive_root = f'{mount_root_abs}/MyDrive'

In [ ]:
import os

# Handbags-to-Shoes Dataset
h2s_root_drive = f'{drive_root}/Datasets/Bags2Shoes'
assert os.path.exists(h2s_root_drive)
h2s_img_zip_abs_drive = f'{h2s_root_drive}/handbags_64.hdf5'

# Test if processed hdf5 file exists in dataset root
assert os.path.exists(h2s_img_zip_abs_drive), f'Please upload the dataset files (handbags_64.hdf5, shoes_64.hdf5) ' + \
                                              f'in Google Drive. \nTried: {h2s_img_zip_abs_drive}'

## 1.2) Clone GitHub repo
Clone achariso/gans-thesis repo into /content/code using git clone.
For more info see: https://medium.com/@purba0101/how-to-clone-private-github-repo-in-google-colab-using-ssh-77384cfef18f

In [ ]:
repo_root = '/content/code/gans-thesis'
!rm -rf "$repo_root"
if not os.path.exists(repo_root) and not os.path.exists(f'{repo_root}/requirements.txt'):
    # Check that ssh keys exist
    assert os.path.exists(f'{drive_root}/GitHub Keys')
    id_rsa_abs_drive = f'{drive_root}/GitHub Keys/id_rsa'
    id_rsa_pub_abs_drive = f'{id_rsa_abs_drive}.pub'
    assert os.path.exists(id_rsa_abs_drive)
    assert os.path.exists(id_rsa_pub_abs_drive)
    # On first run: Add ssh key in repo
    if not os.path.exists('/root/.ssh'):
        # Transfer config file
        ssh_config_abs_drive = f'{drive_root}/GitHub Keys/config'
        assert os.path.exists(ssh_config_abs_drive)
        !mkdir -p ~/.ssh
        !cp -f "$ssh_config_abs_drive" ~/.ssh/
        # # Add github.com to known hosts
        !ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
        # Test: !ssh -T git@github.com

    # Remove any previous attempts
    !rm -rf "$repo_root"
    !mkdir -p "$repo_root"
    # Clone repo
    !git clone git@github.com:achariso/gans-thesis.git "$repo_root"
    src_root = f'{repo_root}/src'
    !rm -rf "$repo_root"/report

## 1.3) Install pip packages
All required files are stored in a requirements.txt files at the repository's root.
Use `pip install -r requirements.txt` from inside the dir to install required packages.

In [ ]:
%cd "$repo_root"
!pip install -r requirements.txt

In [ ]:
# import os
# os.kill(os.getpid(), 9)

In [ ]:
import torch

assert torch.cuda.is_available()

## 1.4) Add code/, */src/ to path
This is necessary in order to be able to run the modules.

In [ ]:
content_root_abs = f'{repo_root}'
src_root_abs = f'{repo_root}/src'
%env PYTHONPATH="/env/python:$content_root_abs:$src_root_abs"
%env NEW_MODEL_NAME__cyclegan=cyclegan_2cb_li5

# 2) Train CycleGAN model on Bags2Shoes_64 dataset
In this section we run the actual training loop for CycleGAN network. CycleGAN consists of two cross-domain generators
and, in our version, two PatchGAN discriminators.

### Colab Bug Workaround
Bug: matplotlib cache not rebuilding.
Solution: Run the following code and then restart the kernel.


In [ ]:
# now inside train_ccgan.py
# os.kill(os.getpid(), 9)

### Actual Run
Eventually, run the code!

In [ ]:
chkpt_step = 'latest'  # supported: 'latest', <int>, None
log_level = 'debug'    # supported: 'debug', 'info', 'warning', 'error', 'critical', 'fatal'
device = 'cuda'        # supported: 'cpu', 'cuda', 'cuda:<GPU_INDEX>'

# Running with -i enables us to get variables defined inside the script (the script runs inline)
%run -i src/train_setup.py --log_level $log_level --chkpt_step $chkpt_step --seed 42 --device $device


### CycleGAN Training

Setup/preparation before starting CycleGAN training loop.


In [ ]:
%cd 'src/'

import torch
from IPython.core.display import display
from torch import Tensor
from torch.nn import DataParallel
# noinspection PyProtectedMember
from torch.utils.data import DataLoader

from datasets.bags2shoes import Bags2ShoesDataset, Bags2ShoesDataloader
from modules.cycle_gan import CycleGAN
from utils.dep_free import get_tqdm
from utils.ifaces import FilesystemDataset
from utils.metrics import GanEvaluator

###################################
###  Hyper-parameters settings  ###
###################################
# TODO: finish this notebook and train in Colab/Kaggle
#   - training
n_epochs = 300
batch_size = 32 if not run_locally else 2
train_test_splits = [90, 10]  # for a 90% training - 10% evaluation set split
#   - evaluation
metrics_n_samples = 1000 if not run_locally else 2
metrics_batch_size = 32 if not run_locally else 1
f1_k = 3 if not run_locally else 1
#   - visualizations / checkpoints steps
display_step = 200
checkpoint_step = 600
metrics_step = 1800  # evaluate model every 3 checkpoints
#   - dataset
target_shape = 64
target_channels = 3
#   - CycleGAN configuration
ccgan_config_id = '64_MSE_L1_L1_2_9_2_9_64_3_1e4_false_false_false'  # as proposed in DiscoGAN paper

###################################
###   Dataset Initialization    ###
###################################
#   - image transforms:
#     If target_shape is different from load one, resize & crop. If target_shape is different from load shape,
#     convert to grayscale.
#     Update: Now done automatically if you set target_channels, target_shape when instantiating the dataloader.
gen_transforms = Bags2ShoesDataset.get_image_transforms(target_shape=target_shape, target_channels=target_channels)
#   - the dataloader used to access the training dataset of cross-scale/pose image pairs at every epoch
#     > len(dataloader) = <number of batches>
#     > len(dataloader.dataset) = <number of total dataset items>
dataloader = Bags2ShoesDataloader(dataset_fs_folder_or_root=datasets_groot, batch_size=batch_size,
                                  image_transforms=gen_transforms, splits=train_test_splits,
                                  pin_memory=not run_locally, count_len_on='max', log_level=log_level)
dataset = dataloader.dataset  # save training dataset as `dataset`
#   - ensure dataset is fetched locally and unzipped
if isinstance(dataset, FilesystemDataset):
    dataset.fetch_and_unzip(in_parallel=False, show_progress=True)
elif hasattr(dataset, 'dataset') and isinstance(dataset.dataset, FilesystemDataset):
    dataset.dataset.fetch_and_unzip(in_parallel=False, show_progress=True)
else:
    raise TypeError('dataset must implement utils.ifaces.FilesystemDataset in order to be auto-downloaded and unzipped')
#   - apply rudimentary tests
assert issubclass(dataloader.__class__, DataLoader)
assert len(dataloader) == len(dataset) // batch_size + (1 if len(dataset) % batch_size else 0)
_bags, _shoes = next(iter(dataloader))
assert tuple(_bags.shape) == (batch_size, target_channels, target_shape, target_shape)
assert tuple(_shoes.shape) == (batch_size, target_channels, target_shape, target_shape)

###################################
###    Models Initialization    ###
###################################
#   - initialize evaluator instance (used to run GAN evaluation metrics: FID, IS, PRECISION, RECALL, F1 and SSIM)
evaluator = GanEvaluator(model_fs_folder_or_root=models_groot, gen_dataset=dataset, target_index=1, device=exec_device,
                         condition_indices=(0,), n_samples=metrics_n_samples, batch_size=metrics_batch_size,
                         f1_k=f1_k)
#   - initialize model
chkpt_step = args.chkpt_step
try:
    if chkpt_step == 'latest':
        ccgan_chkpt_step = chkpt_step
    elif isinstance(chkpt_step, str) and chkpt_step.isdigit():
        ccgan_chkpt_step = int(chkpt_step)
    else:
        ccgan_chkpt_step = None
except NameError:
    ccgan_chkpt_step = None
ccgan = CycleGAN(model_fs_folder_or_root=models_groot, config_id=ccgan_config_id, dataset_len=len(dataset),
                 chkpt_epoch=ccgan_chkpt_step, evaluator=evaluator, device=exec_device, log_level=log_level)
ccgan.logger.debug(f'Using device: {str(exec_device)}')
ccgan.logger.debug(f'Model initialized. Number of params = {ccgan.nparams_hr}')
# FIX: Warmup counters before first batch
if ccgan.step is None:
    ccgan.gforward(batch_size=batch_size)
    ccgan.logger.debug(f'Model warmed-up (internal counters).')
#   - setup multi-GPU training
if torch.cuda.device_count() > 1:
    ccgan.gen = DataParallel(ccgan.gen)
    ccgan.info(f'Using {torch.cuda.device_count()} GPUs for CycleGAN Generator (via torch.nn.DataParallel)')
#   - load dataloader state (from model checkpoint)
if 'dataloader' in ccgan.other_state_dicts.keys():
    dataloader.set_state(ccgan.other_state_dicts['dataloader'])
    ccgan.logger.debug(f'Loaded dataloader state! Current pem_index={dataloader.get_state()["perm_index"]}')

# FIX: Change batch size (if needed)
ccgan.update_batch_size(batch_size, sampler_instance=dataloader.sampler)


### CycleGAN Main training loop

Start/continue training CycleGAN until reaching the desired number of epochs.


In [ ]:
###################################
###       Training Loop         ###
###################################
#   - get the correct tqdm instance
exec_tqdm = get_tqdm()
#   - start training loop from last checkpoint's epoch and step
torch.cuda.empty_cache()
gcapture_ready = True
async_results = None
ccgan.logger.info(f'[training loop] STARTING (epoch={ccgan.epoch}, step={ccgan.initial_step})')
for epoch in range(ccgan.epoch, n_epochs):
    # noinspection PyProtectedMember
    d = {
        'step': ccgan.step,
        'initial_step': ccgan.initial_step,
        'epoch': ccgan.epoch,
        '_counter': ccgan._counter,
        'epoch_inc': ccgan.epoch_inc,
    }
    # initial_step = ccgan.initial_step % len(dataloader)
    ccgan.logger.debug('[START OF EPOCH] ' + str(d))

    bags: Tensor
    shoes: Tensor
    for bags, shoes in exec_tqdm(dataloader, initial=ccgan.initial_step):
        # Transfer image batches to GPU
        bags = bags.to(exec_device)
        shoes = shoes.to(exec_device)

        # Perform a forward + backward pass + weight update on the Generator & Discriminator models
        disc_loss, gen_loss = ccgan(bags, shoes)

        # Metrics & Checkpoint Code
        if ccgan.step % checkpoint_step == 0:
            # Check if another upload is pending
            if not gcapture_ready and async_results:
                # Wait for previous upload to finish
                ccgan.logger.warning('Waiting for previous gcapture() to finish...')
                [r.wait() for r in async_results]
                ccgan.logger.warning('DONE! Starting new capture now.')
            # Capture current model state, including metrics and visualizations
            async_results = ccgan.gcapture(checkpoint=True, metrics=ccgan.step % metrics_step == 0, visualizations=True,
                                           dataloader=dataloader, in_parallel=True, show_progress=True,
                                           delete_after=False)
        # Visualization code
        elif ccgan.step % display_step == 0:
            visualization_img = ccgan.visualize()
            visualization_img.show() if not in_notebook() else display(visualization_img)

        # Check if a pending checkpoint upload has finished
        if async_results:
            gcapture_ready = all([r.ready() for r in async_results])
            if gcapture_ready:
                ccgan.logger.info(f'gcapture() finished')
                if ccgan.latest_checkpoint_had_metrics:
                    ccgan.logger.info(str(ccgan.latest_metrics))
                async_results = None

        # If run locally one pass is enough
        if run_locally and gcapture_ready:
            break

    # If run locally one pass is enough
    if run_locally:
        break

    # noinspection PyProtectedMember
    d = {
        'step': ccgan.step,
        'initial_step': ccgan.initial_step,
        'epoch': ccgan.epoch,
        '_counter': ccgan._counter,
        'epoch_inc': ccgan.epoch_inc,
    }
    ccgan.logger.debug('[END OF EPOCH] ' + str(d))

# Check if a pending checkpoint exists
if async_results:
    ([r.wait() for r in async_results])
    ccgan.logger.info(f'last gcapture() finished')
    if ccgan.latest_checkpoint_had_metrics:
        ccgan.logger.info(str(ccgan.latest_metrics))
    async_results = None

# Training finished!
ccgan.logger.info('[training loop] DONE')


# 3) Evaluate CycleGAN
In this section we evaluate the generation performance of our trained network using the SOTA GAN evaluation metrics.

## 3.1) Get the metrics evolution plots
We plot how the metrics evolved during training. The GAN is **not** trained to minimize those metrics (they are
calculated using `torch.no_grad()`) and thus this evolution merely depends on the network and showcases the correlation
between the GAN evaluation metrics, and the losses (e.g. adversarial & reconstruction) used to optimize the network.

In [ ]:
# Since the CycleGAN implements utils.ifaces.Visualizable, we can
# directly call visualize_metrics() on the model instance.
_ = ccgan.visualize_metrics(upload=True, preview=True)

# Visualize losses (gen & disc in the same plot)
_ = ccgan.visualize_losses(dict_keys=(('gen_loss', 'disc_loss'), ), upload=True, preview=True)

## 3.2) Evaluate Generated Samples
In order to evaluate generated samples and compare model with other GAN architectures trained on the same dataset.
For this purpose we will re-calculate the evaluation metrics as stated above, but with a much bigger number of samples.
In this way, the metrics will be more trustworthy and comparable with the corresponding metrics in the original paper.


In [ ]:
# Initialize a new evaluator instance
# (used to run GAN evaluation metrics: FID, IS, PRECISION, RECALL, F1 and SSIM)
evaluator = GanEvaluator(model_fs_folder_or_root=models_groot, gen_dataset=dataloader.test_set, target_index=1,
                         condition_indices=(0,), n_samples=10000, batch_size=metrics_batch_size,
                         device=exec_device, f1_k=f1_k, ssim_c_img=target_channels)

# Run the evaluator
metrics_dict = evaluator.evaluate(gen=ccgan.gen, metric_name='all', show_progress=True)

# Print results
import json

print(json.dumps(metrics_dict, indent=4))

#
#------------------------------
# Test Set Metrics (epoch=83)
#-----------------------------
#
# {
#     "fid": 66.0643539428711,
#     "is": 4.421818733215332,
#     "f1": 0.7308980226516724,
#     "precision": 0.6072850227355957,
#     "recall": 0.9176950454711914,
#     "ssim": 0.3536342978477478
# }
#


In [ ]:
# Initialize a new evaluator instance
# (used to run GAN evaluation metrics: FID, IS, PRECISION, RECALL, F1 and SSIM)
evaluator = GanEvaluator(model_fs_folder_or_root=models_groot, gen_dataset=dataloader.dataset, target_index=1,
                         condition_indices=(0,), n_samples=10000, batch_size=metrics_batch_size,
                         device=exec_device, f1_k=f1_k, ssim_c_img=target_channels)

# Run the evaluator
metrics_dict = evaluator.evaluate(gen=ccgan.gen, metric_name='all', show_progress=True)

# Print results
import json

print(json.dumps(metrics_dict, indent=4))

#
#----------------------------------
# Training Set Metrics (epoch=83)
#---------------------------------
#
# {
#     "fid": 65.78485870361328,
#     "is": 4.488399982452393,
#     "f1": 0.7314601992022274,
#     "precision": 0.6071811096351624,
#     "recall": 0.9197079261398315,
#     "ssim": 0.3554486334323883
# }
#